<a href="https://colab.research.google.com/github/Samarth1410/ML-Projects/blob/main/resume_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!pip install python-docx

!pip install nltk

!pip install nltk spacy==2.3.5 https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

!pip install pyresparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 13.3 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=49a4f7cc2f7ead3ccda45b0094a123730b0222ad1ecc4f99cfd9fabff38c7da4
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 14.4 MB/s 
     |████████████████████████████████| 10.5 MB 9.4 MB/s 
     |████████████████████████████████| 211 kB 78.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047104 sha256=67fc849a

In [3]:
! python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 12.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import nltk
from pyresparser import ResumeParser
import os
import pandas as pd
import uuid
from docx import Document
import regex as re 
import spacy
import pdfminer
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
from pdfminer.pdfpage import PDFPage
import os 
import docx2txt
import json

In [11]:
nlp = spacy.load("en_core_web_sm")
PHONE_REGEX = r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'
LINKS_REGEX = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def open_pdf_file(file_name):
    ## open the pdf file
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    pagenums = set()
    infile = open(file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()

    result = []

    for line in text.split('\n'):
        line2 = line.strip()
        if line2 != '':
            result.append(line2)
    return (result)

def remove_punctuations(line):
    "remove punctuation"
    return re.sub(r'(\.|\,)', '', line)
  
def preprocess_document(document):
    "preprocess ducument"
    for index, line in enumerate(document):
        line = line.lower()
        line = remove_punctuations(line)
        line = line.split(' ')
       
        document[index] = ' '.join(line)
    return (document)

def pdf_reader(path):
    "pdf reader with preproceoss document"
    document = open_pdf_file(str(path))
    #document = preprocess_document(document)
    text  = ' '
    for x in document:
      text = text + ' ' + x 
    return text


def extract_name(text):
     """
     Name Extractor 
     """
     # Create Doc object
     doc2 = nlp(text)

     # Identify the persons
     persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

     # Return persons
     return persons[0]

def fetch_emails(text):
    """
    Fetch emails

    """
    EMAIL_REGEX = r'[\w\.-]+@[\w\.-]+'
    return re.findall(EMAIL_REGEX, text)[0]

def fetch_phone_numbers(text):
  numbers = re.findall(PHONE_REGEX, text)
  for x in numbers:
    if 10 <=  len(x):
      return x # number

def fetch_links(text):
  return re.findall(LINKS_REGEX,text)

def extract_contact_info_experiences(path):
  """
    Parse Resumes and extract contact info and experience
  """
  try:
    try:
      doc = Document()
      with open(path, 'r') as file:
          doc.add_paragraph(file.read())
      temp_p = uuid.uuid4()
      doc.save("{}.docx".format(temp_p))
      data = ResumeParser("{}.docx".format(temp_p).get_extracted_data())
      
    except:
      data = ResumeParser(path).get_extracted_data()

    return data
  except Exception as e:
    print("ERROR OCCURED",e)
    return ''

def experiances_entity_finder(path):
    "Call the ML Model and load the "
    document = open_pdf_file(path)

    nlp = spacy.load("total_year_and_role__extraction")

    text = ' '
    for x in document:
        text = text + ' '+ x
    doc = nlp(text.lower())
   
    for ent in doc.ents:
        if ent.label_ == 'EXPERIANCE':
            return ent.text

        
def year_extractor(path):
    "EXTRACT YEAR FROM THE ENTITY"
    text = experiances_entity_finder(path)
    """ extrac digit from text"""
    try:
        number = re.findall(r'\d+', text)
    except:
        number= [0]
    return str(number[0]) + ' years' 

In [7]:
nlp = spacy.load("en_core_web_sm")


In [18]:
extract_contact_info_experiences('39365037-Sameh-Samir.pdf')

/usr/local/lib/python3.8/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


By- SAMARTH

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Apoorvgupta[9_0].pdf to Apoorvgupta[9_0].pdf


In [64]:
#EXTRACT RAR FILE FROM COLAB

!pip install patool
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/Resumes (1).rar", outdir="/content/Untitled Folder 1")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
patool: Extracting /content/drive/MyDrive/Resumes (1).rar ...
patool: running /usr/bin/unrar x -- "/content/drive/MyDrive/Resumes (1).rar"
patool:     with cwd='/content/Untitled Folder 1'
patool: ... /content/drive/MyDrive/Resumes (1).rar extracted to `/content/Untitled Folder 1'.


'/content/Untitled Folder 1'

In [65]:
#GET PATH OF ALL THE FILES
import glob
ls = []
for f in glob.glob('/content/Untitled Folder 1/*'):
  ls.append(f)

In [102]:
# CREATE EXCEL SHEET
import openpyxl
wb = openpyxl.Workbook()
ws = wb.active

In [103]:
# EXTRACT INFO FROM THE FILES AND APPEND TO EXCEL

for i in range(len(ls)):
  res = extract_contact_info_experiences(ls[i])
  if len(res)!=0:
    try:
      res['skills'].insert(0, ls[i][27:])
      ws.append(res['skills'])
    except:
      continue


ERROR OCCURED Text extraction is not allowed: <_io.BufferedReader name='/content/Untitled Folder 1/1658258304072.pdf'>
ERROR OCCURED Text extraction is not allowed: <_io.BufferedReader name='/content/Untitled Folder 1/323576571-Ahmed-Zidan-CV.pdf'>


In [104]:
#SAVE EXCEL 
wb.save('output.xlsx')